<a href="https://colab.research.google.com/github/pushkar-khetrapal/EfficientPS/blob/main/EfficientPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image

In [2]:
%%shell

pip install efficientnet_pytorch
pip install git+https://github.com/mapillary/inplace_abn
pip install torch-summary

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=89140f9c859a682bf990c3c0ace8cf62b4d2933a7c287873df0dc9a721a22193
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch
  Cloning https://github.com/mapillary/inplace_abn to /tmp/pip-req-build-1trui6tz
  Running command git clone -q https://github.com/mapillary/inplace_abn /tmp/pip-req-build-1trui6tz
  Created wheel for inplace-abn: filename=inplace_abn-1.1.1.dev1+g845bf23-cp36-cp36m-linux_x86_64.whl size=2745368 sha256=a294c1762d9654a1dd90839b404f51f2f547d0a44eb5f05c84f44c50172e3409
  Stored in directory: /tmp/pip-ephem-wheel-cache-sfn8qwoq/wheels/fe/0b/49/1303ca37166cc1be8784e2367a172133634dcd864a9df0ab56
Successfully built inplace-abn


In [3]:
!git clone https://github.com/pushkar-khetrapal/EfficientPS.git

Cloning into 'EfficientPS'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [4]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

Cloning into 'vision'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 11139 (delta 23), reused 12 (delta 1), pack-reused 11077
Receiving objects: 100% (11139/11139), 12.66 MiB | 21.64 MiB/s, done.
Resolving deltas: 100% (7754/7754), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be37608 version check against PyTorch's CUDA version


In [5]:
from EfficientPS.PretrainedBackbone import TwoWayFPNBackbone

In [6]:
import torchvision
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNN

In [7]:
def get_instance_segmentation_model(num_classes, backbone):
    
    model = MaskRCNN(backbone, num_classes)
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [8]:
from EfficientPS.semantichead import SemanticHead

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as distributed
class EfficientPS(nn.Module):
    def __init__(self):
        super(EfficientPS, self).__init__()

        self.backbone = TwoWayFPNBackbone()

        self.instance = get_instance_segmentation_model(20, self.backbone)

        self.semantic = SemanticHead().to('cuda')
    
    
    def forward(self, img):

        ins = self.instance(img)
        
        x = self.backbone()
        sem = self.semantic(x['3'], x['2'], x['1'], x['0'])
        return [ins, sem]

In [10]:
z = EfficientPS()

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth



Loaded pretrained weights for efficientnet-b5


In [11]:
z

EfficientPS(
  (backbone): FPN(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn0): InPlaceABN(48, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
    (blocks0): Sequential(
      (0): Sequential(
        (0): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (1): InPlaceABN(48, eps=1e-05, momentum=0.1, affine=True, activation=identity)
        (2): Conv2dStaticSamePadding(
          48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (3): InPlaceABN(24, eps=1e-05, momentum=0.1, affine=True, activation=identity)
        (4): MemoryEfficientSwish()
      )
      (1): Sequential(
        (0): Conv2dStaticSamePadding(
          24, 24, kernel_